# Forecasting Agent Experiments

This notebook demonstrates the LSTM forecasting agent and experiments with different configurations.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data import DataLoader, TechnicalIndicators
from src.forecasting_agent import ForecastingAgent
from config import FORECAST_CONFIG

plt.rcParams['figure.figsize'] = (15, 8)

## 1. Prepare Data

In [ ]:
# Load data
ticker = 'AAPL'
loader = DataLoader(ticker, period='2y')
data = loader.fetch_data()

# Add technical indicators
data_with_indicators = TechnicalIndicators.add_all_indicators(data)

print(f"Data shape: {data_with_indicators.shape}")
data_with_indicators.tail()

## 2. Initialize Forecasting Agent

In [ ]:
# Create forecasting agent
config = FORECAST_CONFIG.copy()
config['epochs'] = 20  # Reduce for faster training in notebook

agent = ForecastingAgent(config)
print("Forecasting agent initialized")

## 3. Train Model

In [ ]:
# Split data
train_size = int(len(data_with_indicators) * 0.8)
train_data = data_with_indicators[:train_size]
test_data = data_with_indicators[train_size:]

print(f"Training samples: {len(train_data)}")
print(f"Testing samples: {len(test_data)}")

# Train
agent.train(train_data, validation_split=0.2)

## 4. Generate Forecast

In [ ]:
# Generate forecast
forecast_result = agent.predict(data_with_indicators, forecast_days=10)

print("Forecast Results:")
print("=" * 50)
print(f"Trend: {forecast_result['trend']}")
print(f"Trend Probability: {forecast_result['trend_probability']:.2%}")
print(f"Confidence: {forecast_result['confidence']:.2%}")
print(f"Current Price: ${forecast_result['current_price']:.2f}")
print(f"\nForecast (next {forecast_result['forecast_days']} days):")
for i, (mean, lower, upper) in enumerate(zip(
    forecast_result['mean_forecast'],
    forecast_result['lower_bound'],
    forecast_result['upper_bound']
), 1):
    print(f"Day {i}: ${mean:.2f} (${lower:.2f} - ${upper:.2f})")

## 5. Visualize Forecast

In [ ]:
# Prepare data for plotting
historical_prices = data_with_indicators['Close'].values[-60:]
historical_dates = range(len(historical_prices))

forecast_dates = range(len(historical_prices), 
                       len(historical_prices) + len(forecast_result['mean_forecast']))

# Plot
plt.figure(figsize=(15, 8))

# Historical prices
plt.plot(historical_dates, historical_prices, label='Historical', linewidth=2, color='blue')

# Forecast
plt.plot(forecast_dates, forecast_result['mean_forecast'], 
         label='Forecast', linewidth=2, color='red', linestyle='--')

# Confidence interval
plt.fill_between(forecast_dates, 
                 forecast_result['lower_bound'], 
                 forecast_result['upper_bound'],
                 alpha=0.3, color='red', label='Confidence Interval')

plt.axvline(x=len(historical_prices)-1, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Time Steps')
plt.ylabel('Price ($)')
plt.title(f'{ticker} Price Forecast')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Evaluate Forecast Accuracy

In [ ]:
# Use test data to evaluate
if len(test_data) >= forecast_result['forecast_days']:
    actual_prices = test_data['Close'].values[:forecast_result['forecast_days']]
    predicted_prices = forecast_result['mean_forecast']
    
    # Calculate metrics
    mae = np.mean(np.abs(actual_prices - predicted_prices))
    rmse = np.sqrt(np.mean((actual_prices - predicted_prices) ** 2))
    mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100
    
    print("Forecast Accuracy Metrics:")
    print("=" * 50)
    print(f"MAE:  ${mae:.2f}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    
    # Plot comparison
    plt.figure(figsize=(12, 6))
    x = range(len(actual_prices))
    plt.plot(x, actual_prices, 'o-', label='Actual', linewidth=2)
    plt.plot(x, predicted_prices, 's-', label='Predicted', linewidth=2)
    plt.xlabel('Days')
    plt.ylabel('Price ($)')
    plt.title('Forecast vs Actual Prices')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Not enough test data for evaluation")